In [199]:
import requests
from bs4 import BeautifulSoup
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
load_dotenv()

True

In [200]:
apikey = os.getenv("apikey") #apikey de lastfm

In [201]:
stone_500_rich = pd.read_csv("../data/stone_500_rich.csv")

In [202]:
stone_500_rich.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
115,116,1989,Disintegration,The Cure,Album,4.45,alternative,15984696,687571
361,362,1981,Never Too Much,Luther Vandross,Single,5.00,r&b,814538,167004
117,118,1976,Hotel California,Eagles,Album,4.45,rock,3429786,315961


# Spotify Most streamed
- scrapping

[charset: most streamed albums](https://chartmasters.org/spotify-most-streamed-albums/?y=alltime)
Last updated on 11/06/21

In [203]:
url_spotify = "https://chartmasters.org/spotify-most-streamed-albums/?y=alltime"
html_spotify = requests.get(url_spotify)
soup_spotify = BeautifulSoup(html_spotify.content,"html.parser")

In [204]:
tablas_spotify = soup_spotify.findAll("table")

In [205]:
spotify = tablas_spotify[1]

In [206]:
spotify.find_all("tr")[3] #cuerpo

<tr><td>3</td><td class="Image"><img alt="Dua" height="50" lipa="" src="https://i.scdn.co/image/ab67616d0000b2731764e1a1b94e887206782640" width="50"/></td><td>Dua Lipa</td><td>Dua Lipa (Complete Edition)</td><td class="numbers">8,738,048,733</td><td class="numbers">0</td><td class="numbers">8,723,978</td></tr>

In [207]:
spotify.find_all("th") #cabeceros

[<th>Rank</th>,
 <th>Cover</th>,
 <th>Artist</th>,
 <th>Album</th>,
 <th>Total Streams</th>,
 <th>Daily Streams</th>,
 <th>EAS</th>]

In [208]:
def data_spotify(tab):
    Top_alb = []
    for s in tab.find_all("tr"):
        fila = [elemento for elemento in s.find_all("td")]
        if len(fila)>1:
            spot_dicc = {"Rank" : int(fila[0].text),
                        "Artist": fila[2].text.strip(),
                         "Album": fila[3].text.strip(),
                         "Total" :int(fila[4].text.replace(",","")),
                         "EAS" : int(fila[6].text.replace(",","")) #Equivalent Album Sales
                        }
            Top_alb.append(spot_dicc)
    return Top_alb

In [209]:
tab = spotify
Top_alb_spotify = data_spotify(tab)
Top_alb_spotify[1]

{'Rank': 2,
 'Artist': 'Post Malone',
 'Album': 'beerbongs & bentleys',
 'Total': 9389163476,
 'EAS': 9374044}

In [210]:
Spotify_500 = pd.DataFrame(Top_alb_spotify)
Spotify_500.head(5)

,Rank,Artist,Album,Total,EAS
0,1,Ed Sheeran,÷ (Deluxe),11134771433,11116841
1,2,Post Malone,beerbongs & bentleys,9389163476,9374044
2,3,Dua Lipa,Dua Lipa (Complete Edition),8738048733,8723978
3,4,Post Malone,Hollywood's Bleeding,8625381124,8611492
4,5,Ed Sheeran,x (Wembley Edition),8396308102,8382787


In [227]:
Spotify_500.to_csv("../data/Spotify_500.csv",index = False) 

In [212]:
stone_last_spot= stone_500_rich.merge(Spotify_500, how='left', on = ["Album","Artist"])

In [213]:
stone_last_spot.Rank.fillna(0, inplace=True)
stone_last_spot.Total.fillna(0, inplace=True)
stone_last_spot.EAS.fillna(0, inplace=True)

In [214]:
stone_last_spot[["Rank","Total","EAS"]] = stone_last_spot[["Rank","Total","EAS"]].astype("int64")

In [215]:
stone_last_spot[stone_last_spot["Rank"] != 0].sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners,Rank,Total,EAS
144,145,2000,The Marshall Mathers LP,Eminem,Album,4.20,hip-hop,19346682,1294984,211,2160317727,2156839
471,472,2017,Ctrl,SZA,Album,0.00,r&b,33874763,626859,215,2144313234,2140860
78,79,2016,Blonde,Frank Ocean,Album,4.65,r&b,78748017,859751,133,2647693774,2643430


In [216]:
stone_last_spot.to_csv("../data/Spotify_500.csv",index = False) 

# LastFM 50 Top Artists

In [262]:
url_last50 = f"http://ws.audioscrobbler.com/2.0/?method=chart.gettopartists&api_key={apikey}&format=json"
req_last50 = requests.get(url_last50).json()
last_50 = req_last50["artists"]["artist"]
last_top50 = pd.DataFrame.from_dict(last_50)

In [263]:
last_top50.sample(3)

,name,playcount,listeners,mbid,url,streamable,image
32,Queen,236166768,4666518,420ca290-76c5-41af-999e-564d7c71f1a7,https://www.last.fm/music/Queen,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
46,Playboi Carti,103988284,625308,,https://www.last.fm/music/Playboi+Carti,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
35,Nirvana,266251611,4836780,9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6,https://www.last.fm/music/Nirvana,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [266]:
last_top50.rename(columns={"name": "Artist"}, inplace = True)

In [267]:
last_top50[["playcount","listeners"]] = last_top50[["playcount","listeners"]].astype("int64")


In [268]:
last_top50.to_csv("../data/otras/last_top50_image.csv",index = False) #guardada para mí

In [269]:
last_top50.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Artist      50 non-null     object
 1   playcount   50 non-null     int64 
 2   listeners   50 non-null     int64 
 3   mbid        50 non-null     object
 4   url         50 non-null     object
 5   streamable  50 non-null     object
 6   image       50 non-null     object
dtypes: int64(2), object(5)
memory usage: 2.9+ KB


In [279]:
top_50_last = last_top50[["Artist","playcount","listeners"]]

In [280]:
top_50_last.to_csv("../data/top_50_last.csv",index = False) 

In [281]:
top_50_last.sample(2)

,Artist,playcount,listeners
14,Olivia Rodrigo,74615559,839318
6,Adele,153707605,3431634


In [289]:
stone_500_rich_art = stone_500_rich.groupby("Artist").agg({"Album":"count","Rating":"mean"}).reset_index()

In [293]:
stone_rich_top_50 = stone_500_rich_art.merge(top_50_last,how="left",on = "Artist")


In [295]:
stone_rich_top_50

,Artist,Album,Rating,playcount,listeners
0,2Pac,1,4.650,NaN,NaN
1,50 Cent,1,3.750,NaN,NaN
2,A Tribe Called Quest,2,4.075,NaN,NaN
3,ABBA,1,4.000,NaN,NaN
4,AC/DC,1,4.450,NaN,NaN
...,...,...,...,...,...
332,X,1,5.000,NaN,NaN
333,X-Ray Spex,1,4.000,NaN,NaN
334,Yeah Yeah Yeahs,1,4.650,NaN,NaN
335,Yes,1,4.700,NaN,NaN
